### **PASO 0: Verificar versión de Python**

- **Verificación de la Versión de Python:** Se utiliza `print(sys.version)` para mostrar la versión actual de Python instalada.

In [ ]:
import sys  # Acceder a la información de la versión de Python.
import os  # Manejo de rutas, archivos y operaciones del sistema.
import logging  # Configuración y uso de logs para monitorear la ejecución.

# Configurar la variable de entorno para desactivar la paralelización de tokenizadores y evitar la advertencia de Huggingface
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Configuración de logging para monitorear la ejecución del programa.
logging.basicConfig(
    level=logging.INFO,  # Define el nivel INFO para capturar detalles importantes pero no excesivos.
    format='%(asctime)s - %(levelname)s - %(message)s',  # Especifica un formato estándar para los mensajes de log.
    handlers=[logging.StreamHandler()],  # Envía los logs directamente a la consola.
    force=True  # Forzar la reconfiguración de logging, incluso si ya fue configurado previamente.
)

# Definir la versión requerida de Python
REQUIRED_VERSION = (3, 10, 12)
current_version = sys.version_info

# Validar la versión de Python
if (current_version.major, current_version.minor, current_version.micro) != REQUIRED_VERSION:
    logging.warning("""
    **********************************************
    ** Advertencia: Versión de Python no compatible **
    **********************************************
    Este chatbot está optimizado para Python 3.10.12.
    La versión actual es Python {}.{}.{}.
    Algunas funcionalidades pueden no funcionar correctamente.
    **********************************************
    """.format(current_version.major, current_version.minor, current_version.micro))
else:
    logging.info("""
    **********************************************
    ** Versión de Python compatible **
    **********************************************
    Python 3.10.12 detectado correctamente.
    Todas las funcionalidades deberían operar sin problemas.
    **********************************************
    """)


### **PASO 1: Instalación de Paquetes Necesarios**
Se instalan las bibliotecas necesarias para que el chatbot funcione correctamente.

- **Transformers (`transformers`)**: Para el procesamiento de lenguaje natural.
- **Sentence Transformers (`sentence_transformers`)**: Para crear embeddings eficientes de texto.
- **HNSWlib (`hnswlib`)**: Realiza búsquedas rápidas de vecinos más cercanos.
- **Numpy (`numpy<2.0`)**: Utiliza una versión compatible para operaciones matemáticas.
- **PyPDF2 (`PyPDF2`)**: Manejo y extracción de texto desde archivos PDF.
- **Dotenv (`python-dotenv`)**: Gestiona variables de entorno desde un archivo `.env`.
- **Tenacity (`tenacity`)**: Manejo de reintentos con lógica exponencial.
- **Llama Index (`llama-index` y extensiones para Gemini)**: Proporciona integración con el modelo Gemini.
- **Tqdm (`tqdm`)**: Barra de progreso visual.

In [ ]:
# Instalación de bibliotecas necesarias
%pip install transformers  # Instala la biblioteca Transformers para modelado de lenguaje natural y generación de texto.
%pip install sentence_transformers  # Añade soporte para modelos preentrenados de embeddings de texto.
%pip install hnswlib  # Instala HNSWlib para realizar búsquedas rápidas de vecinos más cercanos.
%pip install numpy<2.0  # Especifica la instalación de una versión compatible de Numpy (menor que 2.0) para evitar conflictos.
%pip install PyPDF2  # Instala PyPDF2 para manejo de archivos PDF y extracción de texto.
%pip install python-dotenv  # Añade soporte para cargar y gestionar variables de entorno desde un archivo `.env`.
%pip install tenacity  # Proporciona herramientas para manejar reintentos de llamadas API con lógica exponencial.
%pip install llama-index  # Instala llama-index, una biblioteca para integrar y estructurar datos con modelos de lenguaje.
%pip install llama-index-llms-gemini  # Extiende llama-index para trabajar con Gemini como modelo de lenguaje.
%pip install llama-index-embeddings-gemini  # Permite generar y utilizar embeddings con el modelo Gemini.
%pip install tqdm  # Añade barras de progreso visuales para mejorar la experiencia del usuario durante procesos largos.
%pip install unidecode  # Normaliza texto eliminando acentos y otros caracteres no ASCII.

### **PASO 2: Importar Librerías y Configurar Logging**
Se importan todas las librerías necesarias y se configura un sistema de logs para monitorear el flujo del programa.

- **Importación de Librerías:** Incluye módulos estándar como `os`, `json`, y `logging`, y bibliotecas específicas del proyecto.
- **Configuración del Logging:** 
  - Configura un formato estándar para los mensajes de log.
  - Establece que los mensajes se impriman directamente en la consola.
  - Define el nivel de logging como `INFO` para capturar detalles esenciales del flujo.
- **Carga de Variables de Entorno:**
  - Usa `load_dotenv()` para cargar claves de API u otras configuraciones sensibles desde un archivo `.env`.

In [ ]:
# Importación de librerías esenciales para el funcionamiento del chatbot.
import os  # Manejo de rutas, archivos y operaciones del sistema.
import json  # Manipulación de datos en formato JSON.
import logging  # Configuración y uso de logs para monitorear la ejecución.
import hnswlib  # Búsqueda eficiente de similitud utilizando índices de alta dimensionalidad.
from transformers import AutoTokenizer, AutoModelForCausalLM  # Modelos preentrenados y tokenizadores para procesamiento de texto.
from sentence_transformers import SentenceTransformer, util  # Embeddings de texto y cálculo de similitud.
import numpy as np  # Operaciones matemáticas avanzadas y estructuras de datos.
from dotenv import load_dotenv  # Carga de variables de entorno desde un archivo `.env`.
from PyPDF2 import PdfReader  # Extracción de texto de documentos PDF.
from tenacity import retry, wait_exponential, stop_after_attempt  # Gestión de reintentos en funciones críticas.
from llama_index.llms.gemini import Gemini  # Interfaz para el modelo de lenguaje Gemini.
from llama_index.core.llms import ChatMessage  # Estructuras de mensajes para interacción con LLMs.
import time  # Manejo de tiempos y medición de duración de procesos.
import hashlib  # Generación de hashes únicos para almacenamiento en caché.
import random  # Generación de valores aleatorios, útil para respuestas personalizadas.
import unicodedata  # Normaliza texto de preguntas

logging.info("Librerías importadas correctamente.")  # Log para confirmar la importación exitosa de todas las librerías.

# Carga de variables de entorno desde un archivo .env para proteger información sensible como claves de API.
load_dotenv()  # Carga las variables de entorno necesarias.
logging.info("Variables de entorno cargadas desde el archivo .env.")  # Log para confirmar que las variables se cargaron correctamente.

### **PASO 3: Cargar Documentos**
Se cargan documentos desde archivos o directorios para analizarlos y extraer contenido relevante.

- **Función `load_documents`:**
  - Permite cargar documentos de varios formatos (`.txt`, `.json`, `.pdf`).
  - Verifica si la fuente existe y lanza un error si no es válida.
  - Itera sobre los archivos en un directorio o procesa un único archivo.
- **Función `extract_content`:**
  - Maneja la lógica para extraer contenido dependiendo del formato:
    - **TXT:** Divide el contenido en bloques usando delimitadores.
    - **JSON:** Carga y devuelve el contenido en formato de diccionario.
    - **PDF:** Extrae texto de todas las páginas utilizando `PdfReader`.

In [ ]:
def normalizar_texto(texto):
    """
    Normaliza el texto eliminando tildes, caracteres especiales, y convirtiendo a minúsculas.
    """
    # Convertir a minúsculas.
    texto = texto.lower()
    # Eliminar tildes y normalizar caracteres unicode.
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Eliminar caracteres que no sean alfanuméricos o espacios.
    texto = ''.join(char for char in texto if char.isalnum() or char.isspace())
    return texto.strip()  # Eliminar espacios extra.

def load_documents(source, is_directory=False):
    """
    Carga documentos desde un archivo o un directorio. Soporta .txt, .json y .pdf.
    """
    loaded_files = []
    if is_directory:
        for filename in os.listdir(source):
            filepath = os.path.join(source, filename)
            if os.path.isfile(filepath) and filepath.endswith(('.txt', '.json', '.pdf')):
                content = extract_content(filepath)
                if content:
                    loaded_files.append({"filename": normalizar_texto(filename), "content": content})
    else:
        content = extract_content(source)
        if content:
            loaded_files.append({"filename": normalizar_texto(os.path.basename(source)), "content": content})
    return loaded_files

def extract_content(filepath):
    """
    Extrae el contenido del archivo según su tipo (.txt, .json, .pdf).
    Si el archivo es .txt, lo divide en unidades por el delimitador '\n-----\n'.
    """
    try:
        if filepath.endswith('.txt'):  # Si es un archivo de texto:
            with open(filepath, 'r', encoding='utf-8') as file:  # Abre el archivo con codificación UTF-8.
                content = file.read()  # Lee el contenido del archivo.
            # Dividir el contenido en unidades por el delimitador
            units = content.split("\n-----\n")  # Divide el texto en bloques por el delimitador.
            return units  # Devuelve una lista de bloques de texto.
        elif filepath.endswith('.json'):  # Si es un archivo JSON:
            with open(filepath, 'r', encoding='utf-8') as file:  # Abre el archivo con codificación UTF-8.
                return json.load(file)  # Carga y retorna el contenido en formato JSON.
        elif filepath.endswith('.pdf'):  # Si es un archivo PDF:
            reader = PdfReader(filepath)  # Instancia el lector PDF.
            return ''.join(page.extract_text() or '' for page in reader.pages)  # Extrae y concatena el texto de todas las páginas.
    except Exception as e:  # Captura cualquier error que ocurra durante la extracción.
        logging.error(f"Error al extraer contenido de '{filepath}': {e}")  # Log del error.
        return None  # Retorna `None` si ocurre un error.

# Cargar documentos (True: busca en directorio, False: busca archivo)
ruta_fuente = 'data'  # Define la ruta del directorio de documentos.
documentos = load_documents(ruta_fuente, is_directory=True)  # Llama a la función para cargar documentos desde el directorio.

# Visualizar cuántos documentos fueron cargados
logging.info(f"Se cargaron {len(documentos)} documentos exitosamente.")  # Log con el número total de documentos cargados.

# Precompute embeddings de nombres de archivos
model_name = "paraphrase-multilingual-MiniLM-L12-v2"  # Modelo para embeddings
model = SentenceTransformer(model_name)

doc_filenames = [doc['filename'] for doc in documentos]
doc_filenames_embeddings = model.encode(doc_filenames, show_progress_bar=True)
logging.info("Embeddings de nombres de archivos precomputados.")


### **PASO 4: Configurar la Clave API de Gemini**
Configura la conexión al modelo Gemini utilizando la clave API proporcionada en un archivo `.env`.

- **Función `configure_gemini`:**
  - Recupera la clave API desde las variables de entorno.
  - Inicializa la instancia de Gemini usando la clave recuperada.
  - Lanza un error si la clave no está configurada correctamente.

In [ ]:
gemini_llm = None  # Variable global para almacenar la instancia del modelo Gemini.

def configure_gemini():
    """
    Configura la instancia de Gemini utilizando la clave API almacenada en variables de entorno.
    """
    global gemini_llm  # Permite modificar la variable global gemini_llm.
    api_key = os.getenv("GEMINI_API_KEY")  # Recupera la clave API desde el archivo .env.
    if not api_key:  # Verifica si la clave API está configurada.
        logging.error("La clave API de Gemini no está configurada.")  # Log de error si no se encuentra la clave.
        raise EnvironmentError("Configura GEMINI_API_KEY en tu archivo .env.")  # Lanza una excepción si la clave no existe.
    gemini_llm = Gemini(api_key=api_key)  # Inicializa una instancia de Gemini con la clave API.
    logging.info("Gemini configurado correctamente.")  # Log para confirmar que la configuración fue exitosa.

# Configurar Gemini
configure_gemini()  # Llama a la función para inicializar y configurar el modelo Gemini.


### **PASO 5: Configurar el Modelo de Embeddings**
Configura el modelo preentrenado de embeddings que se usará para calcular similitudes de texto.

- **Modelo Utilizado:** `"paraphrase-multilingual-MiniLM-L12-v2"`.
- **Carga del Modelo:** Se inicializa con `SentenceTransformer` para generar embeddings.
- **Función `doc_enfermedad`:**
  - Compara embeddings de la pregunta con los nombres de archivos cargados.
  - Devuelve el índice del archivo más relevante basado en la similitud de coseno.

In [ ]:
def doc_enfermedad(pregunta):
    """
    Identifica el archivo de donde leerá la información sobre la enfermedad en la pregunta,
    buscando el mayor embedding entre ella y los nombres de los archivos en documentos.
    """
    try:
        # Generar embedding para la pregunta
        preg_embedding = model.encode(pregunta)
        
        # Calcular similitudes de coseno usando embeddings precomputados
        similarities = [util.cos_sim(preg_embedding, doc_emb).item() for doc_emb in doc_filenames_embeddings]
        
        # Identificar el índice con la mayor similitud
        max_index = similarities.index(max(similarities))
        
        logging.debug(f"Similitudes: {similarities}")
        logging.debug(f"Índice máximo: {max_index} con similitud {similarities[max_index]}")
        
        return max_index
    except Exception as e:
        logging.error(f"Error en doc_enfermedad: {e}")
        return None


### **PASO 6: Crear Clases para Documentos e Índices**
Define clases para manejar documentos y realizar búsquedas eficientes en índices de texto.

- **Clase `Document`:**
  - Representa un documento con contenido (`page_content`) y metadatos.
  - Implementa un método `__str__` para mostrar información relevante del documento.
- **Clase `HNSWIndex`:**
  - Implementa un índice de vecinos más cercanos usando HNSWlib.
  - Admite búsquedas rápidas basadas en similitud de embeddings.

In [ ]:
# Clase para representar documentos cargados con contenido y metadatos.
class Document:
    def __init__(self, text, metadata=None):
        self.page_content = text  # Contenido principal del documento.
        self.metadata = metadata or {}  # Metadatos adicionales, si existen, o un diccionario vacío.
    
    def __str__(self):
        """
        Representación en formato legible de los metadatos del documento.
        Accede a los metadatos de forma segura utilizando `.get()` para evitar errores si faltan claves.
        """
        return (
            f"Título: {self.metadata.get('Title', 'N/A')}\n"
            f"Resumen: {self.metadata.get('Summary', 'N/A')}\n"
            f"Tipo de Estudio: {self.metadata.get('StudyType', 'N/A')}\n"
            f"Paises donde se desarrolla el estudio: {self.metadata.get('Countries', 'N/A')}\n"
            f"Fase en que se encuentra el estudio: {self.metadata.get('Phases', 'N/A')}\n"
            f"Identificación en ClinicaTrial: {self.metadata.get('IDestudio', 'N/A')}.\n\n"
        )

# Clase para manejar índices HNSWlib y realizar búsquedas eficientes de similitud.
class HNSWIndex:
    def __init__(self, embeddings, metadata=None, space='cosine', ef_construction=200, M=16):
        self.dimension = embeddings.shape[1]  # Dimensión de los embeddings.
        self.index = hnswlib.Index(space=space, dim=self.dimension)  # Inicializa el índice HNSW con métrica de coseno.
        self.index.init_index(max_elements=embeddings.shape[0], ef_construction=ef_construction, M=M)  
        # Configura el índice con los parámetros de construcción.
        self.index.add_items(embeddings, np.arange(embeddings.shape[0]))  # Añade los embeddings al índice.
        self.index.set_ef(50)  # Configura el parámetro `ef` para consultas (balance entre velocidad y precisión).
        self.metadata = metadata or []  # Metadatos asociados a los embeddings.
    
    def similarity_search(self, query_vector, k=5):
        """
        Realiza una búsqueda de los `k` elementos más similares al vector de consulta.
        Retorna una lista de tuplas con metadatos y distancias.
        """
        labels, distances = self.index.knn_query(query_vector, k=k)  # Busca los `k` vecinos más cercanos.
        return [(self.metadata[i], distances[0][j]) for j, i in enumerate(labels[0])]  # Asocia los resultados con metadatos.


### **PASO 7: Procesar Documentos y Crear Índices**
Convierte documentos en bloques manejables y genera índices para búsquedas rápidas.

- **Función `desdobla_doc`:**
  - Extrae información relevante de cada documento.
  - Crea embeddings para los textos procesados.
  - Genera un índice HNSWlib con estos embeddings.
- **Proceso por Archivo:** Itera sobre los documentos cargados para crear bloques e índices.

In [ ]:
# Procesar documentos y crear índice
def desdobla_doc(data2):
    """
    Convierte el contenido de los datos proporcionados en instancias de Document y crea un índice HNSWlib.
    """
    documents = []  # Lista para almacenar los objetos Document.
    summaries = []  # Lista para almacenar los resúmenes generados.

    for entry in data2['content']:  # Itera sobre cada entrada en los datos.
        # Extrae información relevante del contenido, manejando valores faltantes con cadenas vacías.
        nctId = entry.get("IDestudio", "")
        briefTitle = entry.get("Title", "")
        summary = entry.get("Summary", "")
        studyType = entry.get("StudyType", "")
        country = entry.get("Countries", "")
        overallStatus = entry.get("OverallStatus", "")
        conditions = entry.get("Conditions", "")
        phases = entry.get("Phases", "")  # Corregir "Phasess" a "Phases"

        # Genera un resumen del estudio utilizando los datos extraídos.
        Summary = (
            f"The study titled '{briefTitle}', of type '{studyType}', "
            f"is being conducted to investigate the condition(s) {conditions}. "
            f"This study is briefly summarized as follows: {summary}. "
            f"Currently, the study status is {overallStatus}, and it is taking place in {country}. "
            f"The study is classified under {phases} phase. "
            f"For more information, search {nctId} on ClinicalTrials."
        )

        # Asocia el resumen con sus metadatos.
        metadata = {"Summary": Summary}

        # Crea una instancia de Document y la agrega a la lista.
        documents.append(Document(Summary, metadata))
        summaries.append(Summary)  # Almacena el resumen para posibles referencias.

    # Genera embeddings para todos los documentos utilizando el modelo cargado.
    embeddings = model.encode([doc.page_content for doc in documents], show_progress_bar=True)
    embeddings = np.array(embeddings).astype(np.float32)  # Convierte los embeddings a float32 para compatibilidad.

    # Crea un índice HNSWlib utilizando los embeddings generados.
    vector_store = HNSWIndex(embeddings, metadata=[doc.metadata for doc in documents])

    return documents, vector_store  # Retorna los documentos procesados y el índice creado.

# Genera trozos e índices para las enfermedades y los almacena en listas.
trozos_archivos = []  # Lista para almacenar los bloques procesados de documentos.
index_archivos = []  # Lista para almacenar los índices HNSWlib.

for i in range(len(documentos)):  # Itera sobre cada conjunto de documentos.
    trozos, index = desdobla_doc(documentos[i])  # Procesa y crea el índice para cada conjunto.
    trozos_archivos.append(trozos)  # Almacena los bloques procesados.
    index_archivos.append(index)  # Almacena el índice creado.

logging.info("Índices HNSWlib creados para todos los documentos.")  # Log para confirmar la creación exitosa de los índices.

### **PASO 8: Traducir Preguntas y Respuestas**
Se encarga de traducir texto entre idiomas usando el modelo Gemini.

- **Función `traducir`:**
  - Envía el texto a traducir como un mensaje al modelo Gemini.
  - Retorna la traducción y registra el tiempo tomado para la operación.

In [ ]:
def traducir(texto, idioma_destino):
    """
    Traduce texto al idioma especificado utilizando el modelo Gemini.
    """
    start_time = time.time()  # Inicia el contador para medir el tiempo de traducción.

    # Prepara los mensajes para enviar al modelo de lenguaje.
    mensajes = [
        ChatMessage(role="system", content="Actúa como un traductor."),
        ChatMessage(role="user", content=f"Por favor, traduce este texto al {idioma_destino}: {texto}")
    ]
    try:
        respuesta = gemini_llm.chat(mensajes)
        elapsed_time = time.time() - start_time  # Calcula el tiempo transcurrido.
        logging.info(f"Traducción completada: {respuesta.message.content.strip()} en {elapsed_time:.2f} segundos.")
        return respuesta.message.content.strip()  # Devuelve la traducción limpia.
    except Exception as e:
        logging.error(f"Error al traducir: {e}")  # Registra el error en los logs.
        return texto  # Devuelve el texto original como fallback.

def generate_embedding(texto):
    """
    Genera un embedding para una pregunta o texto.
    """
    try:
        texto_normalizado = normalizar_texto(texto)
        embedding = model.encode([texto_normalizado])
        logging.info(f"Embedding generado para el texto: {texto_normalizado}")
        return embedding
    except Exception as e:
        logging.error(f"Error al generar el embedding: {e}")
        return np.zeros((1, 768))  # Ajusta el tamaño según tu modelo

def obtener_contexto(pregunta, index, trozos, top_k=50):
    """
    Recupera los trozos de texto más relevantes para responder la pregunta.
    """
    try:
        pregunta_normalizada = normalizar_texto(pregunta)
        pregunta_en_ingles = traducir(pregunta_normalizada, "inglés")
        if not pregunta_en_ingles or len(pregunta_en_ingles) < 5:  # Validación básica.
            logging.warning("Traducción fallida o insuficiente.")
            return "No se pudo procesar la pregunta debido a un error en la traducción."
        pregunta_emb = generate_embedding(pregunta_en_ingles)

        # Buscar en el índice
        results = index.similarity_search(pregunta_emb, k=top_k)
        if not results:
            logging.warning("No se encontró contexto relevante.")
            return "No se encontró información relevante para esta pregunta."

        texto = "\n".join([entry[0]["Summary"] for entry in results])
        return texto
    except Exception as e:
        logging.error(f"Error al obtener el contexto: {e}")
        return "Hubo un problema al recuperar la información. Por favor, intenta con otra pregunta."


### **PASO 9: Generar Respuestas**
Genera respuestas específicas basadas en la pregunta del usuario y el contexto recuperado.

- **Función `categorizar_pregunta`:**
  - Clasifica la pregunta en categorías como "tratamiento", "ensayo", etc.
- **Función `generar_prompt`:**
  - Crea prompts personalizados según la categoría.
- **Función `es_saludo`:**
  - Detecta si la entrada del usuario es un saludo.
- **Función `responder_saludo`:**
  - Devuelve una respuesta aleatoria de saludo.

In [ ]:
# Mejorar la Generación de Respuestas con Prompts Específicos
def categorizar_pregunta(pregunta):
    """
    Clasifica la pregunta en categorías específicas como 'tratamiento', 'ensayo clínico', etc.
    """
    categorias = {
        "tratamiento": ["tratamiento", "medicación", "cura", "terapia", "fármaco", "intervención", "intervenciones"],
        "ensayo": ["ensayo", "ensayos", "estudio", "estudios", "prueba", "investigación", "trial"],
        "criterios": ["criterios", "inclusión", "exclusión", "participantes"],
        "resultado": ["resultado", "efectividad", "resultados", "éxito", "fracaso"],
        "ubicación": ["ciudad", "ciudades", "país", "países", "ubicación", "localización"],
        "prevención": ["prevención", "previene", "evitar", "reducción de riesgo"],
        "duración": ["duración", "años", "meses", "plazo"],
    }
    for categoria, palabras in categorias.items():
        if any(palabra in pregunta.lower() for palabra in palabras):
            return categoria
    return "general"

def generar_prompt(categoria, pregunta):
    """
    Genera un prompt específico basado en la categoría de la pregunta.
    """
    prompts = {
        "tratamiento": f"Proporciona información detallada sobre tratamientos relacionados con: {pregunta}.",
        "ensayo": f"Describe los ensayos clínicos actuales relacionados con: {pregunta}.",
        "criterios": f"Explica los criterios de inclusión y exclusión para los ensayos clínicos sobre: {pregunta}.",
        "resultado": f"Explica los resultados más recientes de ensayos clínicos sobre: {pregunta}.",
        "ubicación": f"Indica las ubicaciones geográficas donde se están llevando a cabo ensayos clínicos para: {pregunta}.",
        "prevención": f"Ofrece estrategias de prevención para: {pregunta}.",
        "duración": f"Describe la duración típica de los ensayos clínicos sobre: {pregunta}.",
    }
    return prompts.get(categoria, "Por favor, responde la pregunta sobre ensayos clínicos.")

# Se define una función para detectar si una pregunta es un saludo.
def es_saludo(pregunta):
    saludos = ["hola", "buen día", "buenas", "cómo estás", "cómo te llamas", "qué tal", "estás bien", "buenas tardes", "buenas noches"]
    return any(saludo in pregunta.lower() for saludo in saludos)

def responder_saludo():
    saludos_respuestas = [
        "¡Hola! Estoy para ayudarte con información sobre ensayos clínicos. ¿En qué puedo asistirte hoy?",
        "¡Buenas! Tenés alguna pregunta sobre ensayos clínicos en enfermedades neuromusculares?",
        "¡Hola! ¿Cómo puedo ayudarte con tus consultas sobre ensayos clínicos?"
    ]
    return random.choice(saludos_respuestas)

def generar_respuesta(pregunta, contexto, prompt_especifico):
    """
    Genera una respuesta usando el contexto proporcionado y un prompt específico.
    """
    start_time = time.time()  # Medir el tiempo de generación de respuesta

    mensajes = [
        ChatMessage(role="system", content="Eres un experto médico."),
        ChatMessage(role="user", content=f"{prompt_especifico}\nContexto: {contexto}\nPregunta: {pregunta}")
    ]
    try:
        respuesta = gemini_llm.chat(mensajes)
        elapsed_time = time.time() - start_time  # Tiempo de generación de respuesta
        logging.info(f"Respuesta generada en inglés: {respuesta.message.content.strip()} en {elapsed_time:.2f} segundos.")

        # Traducir la respuesta al español
        respuesta_en_espanol = traducir(respuesta.message.content, "español")
        logging.info(f"Respuesta traducida al español: {respuesta_en_espanol}")
        return respuesta_en_espanol
    except Exception as e:
        logging.error(f"Error al generar la respuesta: {e}")
        return "Lo siento, ocurrió un error al generar la respuesta."

### **PASO 10: Función Principal para Responder Preguntas**
Integra todos los pasos anteriores para generar respuestas completas.

- **Función `obtener_respuesta_cacheada`:**
  - Verifica si una pregunta ya tiene respuesta en caché.
- **Función `guardar_respuesta_cacheada`:**
  - Almacena la respuesta generada en caché.
- **Función `responder_pregunta`:**
  - Traduce la pregunta.
  - Recupera el contexto relevante.
  - Genera la respuesta y la almacena en caché.

In [ ]:
def generar_hash(pregunta):
    """
    Genera un hash SHA-256 único para la pregunta proporcionada.
    Este hash se utiliza como identificador para el almacenamiento en caché.
    """
    return hashlib.sha256(pregunta.encode('utf-8')).hexdigest()  # Convierte la pregunta a un hash hexadecimal.

def obtener_respuesta_cacheada(pregunta):
    """
    Recupera una respuesta previamente generada desde el caché, si existe.
    """
    hash_pregunta = generar_hash(pregunta)  # Genera el hash único para la pregunta.
    archivo_cache = f"cache/{hash_pregunta}.json"  # Define la ruta del archivo de caché.
    if os.path.exists(archivo_cache):  # Verifica si el archivo de caché existe.
        try:
            with open(archivo_cache, "r", encoding='utf-8') as f:  # Abre el archivo en modo lectura.
                datos = json.load(f)  # Carga los datos en formato JSON.
                return datos.get("respuesta", None)  # Retorna la respuesta si está disponible.
        except Exception as e:  # Manejo de errores al leer el archivo.
            logging.error(f"Error al leer el caché para la pregunta '{pregunta}': {e}")
            return None  # Retorna None en caso de error.
    return None  # Retorna None si el archivo de caché no existe.

def guardar_respuesta_cacheada(pregunta, respuesta):
    """
    Almacena una respuesta en el caché para consultas futuras.
    """
    hash_pregunta = generar_hash(pregunta)  # Genera el hash único para la pregunta.
    archivo_cache = f"cache/{hash_pregunta}.json"  # Define la ruta del archivo de caché.
    try:
        os.makedirs(os.path.dirname(archivo_cache), exist_ok=True)  # Asegura que el directorio del caché exista.
        with open(archivo_cache, "w", encoding='utf-8') as f:  # Abre el archivo en modo escritura.
            json.dump({"pregunta": pregunta, "respuesta": respuesta}, f, ensure_ascii=False, indent=4)  
            # Guarda la pregunta y la respuesta en formato JSON.
        logging.info(f"Respuesta cacheada para la pregunta: '{pregunta}'")  # Log de éxito al guardar en caché.
    except Exception as e:  # Manejo de errores al guardar en caché.
        logging.error(f"Error al guardar la respuesta en caché para la pregunta '{pregunta}': {e}")

def responder_pregunta(pregunta, index, trozos):
    """
    Responde una pregunta del usuario integrando:
    - Búsqueda en caché.
    - Traducción y recuperación de contexto.
    - Generación de respuestas personalizadas.

    Incluye manejo de caché para optimizar el tiempo de respuesta.
    """
    try:
        # Verificar si la respuesta ya está en el caché.
        respuesta_cacheada = obtener_respuesta_cacheada(pregunta)
        if respuesta_cacheada:
            logging.info(f"Respuesta obtenida del caché para la pregunta: '{pregunta}'")
            return respuesta_cacheada

        # Categorizar la pregunta para generar prompts específicos.
        categoria = categorizar_pregunta(pregunta)
        logging.info(f"Categoría de la pregunta: {categoria}")

        # Generar un prompt basado en la categoría.
        prompt_especifico = generar_prompt(categoria, pregunta)
        logging.info(f"Prompt específico generado: {prompt_especifico}")

        # Obtener el contexto relevante para la pregunta.
        contexto = obtener_contexto(pregunta, index, trozos)
        logging.debug(f"Contexto recuperado: {contexto[:200] if contexto else 'Sin contexto'}")
        if not contexto:
            logging.warning("No se encontró un contexto relevante para la pregunta.")
            respuesta = "No pude encontrar información relevante para responder tu pregunta."
            guardar_respuesta_cacheada(pregunta, respuesta)
            return respuesta

        # Generar la respuesta final utilizando el contexto y el prompt.
        respuesta = generar_respuesta(pregunta, contexto, prompt_especifico)
        logging.debug(f"Respuesta generada: {respuesta[:200] if respuesta else 'Sin respuesta'}")

        # Guardar la respuesta generada en el caché.
        guardar_respuesta_cacheada(pregunta, respuesta)

        return respuesta
    except Exception as e:
        logging.error(f"Error en el proceso de responder pregunta: {e}")
        return "Ocurrió un error al procesar tu pregunta."

### **PASO 11: Interfaz CLI**
Proporciona una interfaz interactiva en línea de comandos para que los usuarios interactúen con el chatbot.

- **Inicialización del Caché:** Crea el directorio de caché si no existe.
- **Bucle Principal:**
  - Permite al usuario ingresar preguntas.
  - Responde saludos inmediatamente si se detectan.
  - Recupera el índice y bloques relacionados con la enfermedad mencionada.
  - Genera y muestra la respuesta al usuario.

In [ ]:
if __name__ == "__main__":
    # Asegurar que el directorio de caché exista
    os.makedirs("cache", exist_ok=True)  # Crea el directorio "cache" si no existe para almacenar respuestas cacheadas.
    
    # Mensaje de bienvenida e instrucciones para el usuario
    print("Bienvenido al chatbot de ensayos clínicos")
    print("Conversemos sobre Ensayos Clínicos\n de las siguientes enfermedades neuromusculares: Distrofia Muscular de Duchenne o de Becker, Enfermedad de Pompe, Distrofia Miotónica o Enfermedad de almacenamiento de glucógeno")
    print("Escribe tu pregunta, dejando claramente expresada la enfermedad sobre la que quieres conocer información de ensayos médicos. Escribe 'salir' para terminar.")
    
    while True:
        # Solicitar la pregunta al usuario
        pregunta = input("Tu pregunta: ").strip()  # Recoge la entrada del usuario y elimina espacios innecesarios.

        # Verificar si el usuario desea salir
        if pregunta.lower() in ['salir', 'chau', 'exit', 'quit']:  # Frases que permiten terminar la sesión.
            print("¡Chau!")  # Mensaje de despedida.
            logging.info("El usuario ha finalizado la sesión.")  # Registra la finalización de la sesión en los logs.
            break  # Sale del bucle principal.

        # Verificar si la entrada es un saludo
        if es_saludo(pregunta):
            respuesta_saludo = responder_saludo()
            print(respuesta_saludo)
            logging.info("Se detectó un saludo del usuario.")
            continue  # Salta al siguiente ciclo del bucle.

        # Normalizar la pregunta
        pregunta_normalizada = normalizar_texto(pregunta)
        logging.info(f"Pregunta procesada: {pregunta_normalizada}")
        logging.debug(f"Pregunta original: {pregunta}")

        # Identificar la enfermedad relacionada con la pregunta
        idn = doc_enfermedad(pregunta_normalizada)
        if idn is None:
            logging.warning(f"No se pudo identificar la enfermedad para la pregunta: '{pregunta}'")
            respuesta = "Lo siento, no pude identificar la enfermedad relacionada con tu pregunta."
            print(f"Respuesta: {respuesta}")
            continue

        try:
            index = index_archivos[idn]
            trozos = trozos_archivos[idn]

            # Generar la respuesta para la pregunta del usuario
            respuesta = responder_pregunta(pregunta_normalizada, index, trozos)
        except Exception as e:
            logging.error(f"Error al generar la respuesta para la pregunta '{pregunta}': {e}")
            respuesta = "Lo siento, ocurrió un error al generar la respuesta."

        print(f"Respuesta: {respuesta}")

### Anexo: Generar respuestas desde una lista de preguntas y guardar en un archivo TXT**
Este paso procesa una lista de preguntas predefinidas, genera respuestas y las guarda en un archivo de salida.

In [ ]:
import unicodedata
import logging

# Configurar logging
logging.basicConfig(level=logging.DEBUG)  # Asegúrate de que el nivel de logging esté configurado para capturar DEBUG

# Función para normalizar texto
def normalizar_texto(texto):
    """
    Normaliza el texto eliminando tildes, caracteres especiales, y convirtiendo a minúsculas.
    """
    texto = texto.lower()  # Convertir a minúsculas.
    # Eliminar tildes y normalizar caracteres unicode.
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    # Eliminar caracteres que no sean alfanuméricos o espacios.
    texto = ''.join(char for char in texto if char.isalnum() or char.isspace())
    return texto.strip()  # Eliminar espacios extra.

# Lista de sustituciones para problemas comunes
sustituciones = {
    "ensayos": "estudios",
    "ó": "o",
    "á": "a",
    "í": "i",
    "ú": "u",
    "é": "e",
    "Distrofia Muscular de Duchenne": "Duchenne",
    "Distrofia Muscular de Becker": "Becker",
    "Enfermedad de Pompe": "Pompe",
    "Distrofia Miotónica": "Miotónica",
    "Enfermedad de almacenamiento de glucógeno": "Glucógeno"
}

def corregir_pregunta(pregunta):
    """
    Realiza sustituciones en la pregunta para evitar problemas en la búsqueda.
    """
    for palabra, reemplazo in sustituciones.items():
        pregunta = pregunta.replace(palabra, reemplazo)
    return pregunta

# Lista de preguntas hardcodeadas
preguntas = [
    "¿Cuántos ensayos clínicos están activos actualmente para la Distrofia Muscular de Duchenne?",
    "¿Qué tratamientos están siendo investigados para la Enfermedad de Pompe?",
    "¿Cuáles son las fases de los ensayos clínicos disponibles para la Distrofia Miotónica?",
    "¿Existen estudios completados sobre la Enfermedad de almacenamiento de glucógeno? Si es así, ¿qué resultados destacaron?",
    "¿Qué países participan en los ensayos clínicos relacionados con la Distrofia Muscular de Becker?",
    "¿Se está investigando alguna terapia génica para la Distrofia Muscular de Duchenne?",
    "¿Qué medicamentos están siendo evaluados en ensayos clínicos para mejorar la fuerza muscular en la Enfermedad de Pompe?",
    "¿Existen ensayos clínicos que investiguen Litifilimab (BIIB059) en la Distrofia Miotónica?",
    "¿Qué tipo de intervenciones se están probando en los estudios para la Enfermedad de almacenamiento de glucógeno?",
    "¿Hay estudios que comparen la eficacia de diferentes tratamientos en la Distrofia Muscular de Becker?",
    "¿Cuáles son los criterios de inclusión para los ensayos clínicos sobre la Distrofia Muscular de Duchenne?",
    "¿Hay ensayos clínicos diseñados específicamente para niños con Enfermedad de Pompe?",
    "¿Qué estudios incluyen pacientes adultos con Distrofia Miotónica?",
    "¿Hay ensayos clínicos que evalúen tratamientos preventivos para la Enfermedad de almacenamiento de glucógeno?",
    "¿Existen ensayos multicéntricos en América Latina para estas enfermedades?",
    "¿Cuántos ensayos clínicos sobre la Enfermedad de Pompe se están llevando a cabo en Argentina?",
    "¿Qué estudios tienen una duración estimada de más de 5 años en la Distrofia Muscular de Duchenne?",
    "¿Qué ciudades participan en los ensayos clínicos para la Distrofia Muscular de Becker en Europa?",
    "¿Hay ensayos clínicos de la Enfermedad de almacenamiento de glucógeno en Canadá?",
    "¿Cuáles son las instituciones que lideran ensayos clínicos en Estados Unidos para la Distrofia Miotónica?",
]

# Nombre del archivo de salida
archivo_salida = "respuestas.txt"

# Abrir el archivo en modo de escritura
with open(archivo_salida, "w", encoding="utf-8") as file:
    for pregunta in preguntas:
        # Paso 1: Corregir y normalizar la pregunta
        pregunta_corregida = corregir_pregunta(pregunta)
        pregunta_normalizada = normalizar_texto(pregunta_corregida)
        
        # Log para depuración
        logging.info(f"Pregunta procesada: {pregunta_normalizada}")
        logging.debug(f"Pregunta original: {pregunta}")
        
        # Paso 2: Determinar si es un saludo
        if es_saludo(pregunta_normalizada):
            respuesta = responder_saludo()
        else:
            # Identificar el índice del documento
            try:
                idn = doc_enfermedad(pregunta_normalizada)
                if idn is None:
                    raise ValueError("No se pudo identificar la enfermedad relacionada con la pregunta.")
                index = index_archivos[idn]
                trozos = trozos_archivos[idn]

                # Generar respuesta
                respuesta = responder_pregunta(pregunta_normalizada, index, trozos)
            except Exception as e:
                logging.error(f"Error al procesar la pregunta '{pregunta_normalizada}': {e}")
                respuesta = "Lo siento, no pude procesar tu pregunta."

        # Escribir pregunta y respuesta en el archivo
        file.write(f"Pregunta: {pregunta}\n")
        file.write(f"Respuesta: {respuesta}\n\n")

# Confirmar que se han guardado las respuestas
logging.info(f"Respuestas guardadas en el archivo '{archivo_salida}'.")
print(f"Respuestas guardadas en el archivo '{archivo_salida}'.")